# 각종 라이브러리 설치 및 Yolov8 설치

In [1]:
# 프로젝트 경로 이동
import os
os.chdir("/content/drive/MyDrive/line_detection")

In [ ]:
!git clone http://github.com/ultralytics/ultralytics.git

In [ ]:
!pip install tqdm
!pip install torch torchvision numpy matplotlib
!pip install ultralytics
!pip install -r requirements.txt
!yolo seg
!pip install ultralytics opencv-python

# 라이브러리

In [ ]:
from tqdm import tqdm
import zipfile
import os
from ultralytics import YOLO
import shutil
import random
import cv2
import numpy as np
import yaml
import matplotlib.pyplot as plt
from IPython.display import Audio, display
from glob import glob
from google.colab.patches import cv2_imshow
from IPython.display import display, clear_output
import PIL.Image
from google.colab import files

In [ ]:
!pip freeze > requirements.txt # yolov8n.pt 파일 생성 확인
!yolo predict

# 데이터셋 생성 과정

In [ ]:
# 데이터 폴더 경로
image_folder = '/content/drive/MyDrive/line_detection/images'
label_folder = '/content/drive/MyDrive/line_detection/labels'

# 나눌 폴더 경로
working_dir = '/content/drive/MyDrive/line_detection'
train_folder = os.path.join(working_dir, 'train')
valid_folder = os.path.join(working_dir, 'valid')
test_folder = os.path.join(working_dir, 'test')

# 폴더 생성
for folder in [train_folder, valid_folder, test_folder]:
    os.makedirs(os.path.join(folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(folder, 'labels'), exist_ok=True)

# 이미지와 라벨 파일 리스트 가져오기
images = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))])
labels = sorted([f for f in os.listdir(label_folder) if f.endswith(('.txt'))])

# 이미지 파일의 수에 따라 데이터 분할 비율을 설정
num_images = len(images)
num_train = int(num_images * 0.8)
num_valid = int(num_images * 0.1)
num_test = num_images - num_train - num_valid

# 파일을 섞어줍니다
random.shuffle(images)

# 데이터 나누기
train_images = images[:num_train]
valid_images = images[num_train:num_train + num_valid]
test_images = images[num_train + num_valid:]

def copy_files(file_list, src_folder, dst_folder):
    for file_name in file_list:
        image_src = os.path.join(src_folder, file_name)
        label_src = os.path.join(label_folder, file_name.replace('.jpg', '.txt').replace('.png', '.txt'))

        dst_image = os.path.join(dst_folder, 'images', file_name)
        dst_label = os.path.join(dst_folder, 'labels', file_name.replace('.jpg', '.txt').replace('.png', '.txt'))

        # 파일 복사
        shutil.copy(image_src, dst_image)
        shutil.copy(label_src, dst_label)

# 파일 복사
copy_files(train_images, image_folder, train_folder)
copy_files(valid_images, image_folder, valid_folder)
copy_files(test_images, image_folder, test_folder)

print("Data split completed.")


In [ ]:
!pwd
train_img_list = glob('/content/drive/MyDrive/line_detection/train/images/*.png')
train_txt_list = glob('/content/drive/MyDrive/line_detection/train/labels/*.txt')

valid_img_list = glob('/content/drive/MyDrive/line_detection/valid/images/*.png')
valid_txt_list = glob('/content/drive/MyDrive/line_detection/valid/labels/*.txt')

test_img_list = glob('/content/drive/MyDrive/line_detection/test/images/*.png')
test_txt_list = glob('/content/drive/MyDrive/line_detection/test/labels/*.txt')

print(len(train_img_list), len(train_txt_list))
print(len(valid_img_list), len(valid_txt_list))
print(len(test_img_list), len(test_txt_list))

#데이터 사용 경로 및 방법

In [ ]:
# 파일 사용 , train.txt , val.txt 파일 생성
with open('/content/drive/MyDrive/line_detection/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/line_detection/val.txt', 'w') as f:
    f.write('\n'.join(valid_img_list) + '\n')


In [ ]:
# yaml 파일 수정하기
with open('/content/drive/MyDrive/line_detection/data.yaml', 'r') as f :
  data = yaml.full_load(f) # yaml.load -> yaml.full_load

print(data)

# 경로 설정하기
data['train'] = '/content/drive/MyDrive/line_detection/train.txt'
data['val'] = '/content/drive/MyDrive/line_detection/val.txt'


# 학습

In [ ]:
#yolo v8 모델을 사용하여 segmentation 수행
!pwd
!yolo task=segment mode=train model=yolov8s-seg.pt data=/content/drive/MyDrive/line_detection/data.yaml epochs=50  lr0=0.001 batch=32 device=0 dropout=0.2

# 결과 분석

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 영상 테스트 진행

In [ ]:
""" # pre-trained 모델 호출
model = YOLO('/content/drive/MyDrive/line_detection/yolov8s-seg.pt')

# 사용자 지정 학습된 모델 경로 설정
best_model_path = os.path.join('/content/drive/MyDrive/line_detection/runs/segment/train2/weights/best.pt')

# 사용자 지정 학습된 모델 호출
best_model = YOLO(best_model_path)
"""

In [ ]:
# pre-trained 모델 호출
model = YOLO('/content/drive/MyDrive/line_detection/yolov8s-seg.pt')

# 사용자 지정 학습된 모델 경로 설정
best_model_path = os.path.join('/content/drive/MyDrive/line_detection/runs/segment/train2/weights/best.pt')

# 사용자 지정 학습된 모델 호출
best_model = YOLO(best_model_path)



In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np

try:
    # YOLO 모델 로드
    model = YOLO('/content/drive/MyDrive/line_detection/runs/segment/train2/weights/best.pt')

    # 비디오 파일 열기
    video = cv2.VideoCapture('/content/drive/MyDrive/line_detection/Video_Dataset/Test_Video(1).avi')

    if not video.isOpened():
        raise IOError("비디오 파일을 열 수 없습니다.")

    # 출력 비디오 설정
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)훼
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/content/drive/MyDrive/line_detection/Video_Dataset/Test_Video(a).avi', fourcc, fps, (width, height))

    # 색상 맵 설정 (2개의 클래스: 빨강, 파랑)
    color_map = np.array([[255, 0, 0], [0, 0, 255]])  # 빨강, 파랑

    frame_count = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break

        # YOLO로 세그멘테이션 수행
        results = model(frame)

        # 세그멘테이션 마스크 처리
        if results[0].masks is not None:
            for i, (mask, box, cls) in enumerate(zip(results[0].masks.xy, results[0].boxes.xyxy, results[0].boxes.cls)):
                color = color_map[int(cls) % len(color_map)]
                pts = np.array(mask, np.int32)
                pts = pts.reshape((-1, 1, 2))

                # 마스크 그리기 (반투명)
                overlay = frame.copy()
                cv2.fillPoly(overlay, [pts], color.tolist())
                cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)  # 투명도 조절

                # 바운딩 박스 그리기
                x1, y1, x2, y2 = map(int, box[:4])
                cv2.rectangle(frame, (x1, y1), (x2, y2), color.tolist(), 2)

                # 클래스 이름과 정확도 표시
                label = f"{results[0].names[int(cls)]}: {results[0].boxes.conf[i]:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color.tolist(), 2)

        # 결과 저장
        out.write(frame)

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"{frame_count} 프레임 처리 완료")

    print("비디오 처리 완료")

except Exception as e:
    print(f"오류 발생: {str(e)}")

finally:
    video.release()
    out.release()
    cv2.destroyAllWindows()